In [ ]:
import module_loader
import pandas as pd
from modules.curves import *
from modules.dual import Dual

### Define our Market Rates and Solve our Curve

We will create 1y, 2y, 5y and 10y par swap rates

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2023, 1, 1): Dual(1, {"v1": 1}),
    datetime(2024, 1, 1): Dual(1, {"v2": 1}),
    datetime(2027, 1, 1): Dual(1, {"v3": 1}),
    datetime(2032, 1, 1): Dual(1, {"v4": 1})
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 12, 12): 1.210,
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.635,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.885,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.930,
}
s_cv = SolvedCurve(
    nodes=nodes,
    swaps=list(swaps.keys()),
    obj_rates=list(swaps.values()),
    interpolation="log_linear",
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

Check that our solved curve is re-pricing our input swap rates

In [ ]:
for swap in swaps.keys():
    print(swap.rate(s_cv).real)

In [ ]:
for swap in swaps.keys():
    print(swap.analytic_delta(s_cv).real)

Yes, close enough!

Lets check that our risk function generates risk only for the exact instrument

In [ ]:
risk = {}
for swap in swaps.keys():
    risk.update({swap.end: swap.risk(s_cv, fixed_rate=swap.rate(s_cv).real)[:, 0]})
    
df = pd.DataFrame(risk, index=["1y", "2y", "5y", "10y"])
df.style.format("{:.3f}")

That is close enough with tolerance. 

### Forward to Par Jacobian

Now lets consider some forward swaps, and update their rates with our solved curve, and risk these swaps against our curve.

In [ ]:
fwd_swaps = {
    Swap(datetime(2022, 1, 1), 12*1, 12, 12): 1,
    Swap(datetime(2023, 1, 1), 12*1, 12, 12): 1,
    Swap(datetime(2024, 1, 1), 12*3, 12, 12): 1,
    Swap(datetime(2027, 1, 1), 12*5, 12, 12): 1,
}
for swap in fwd_swaps.keys():
    fwd_swaps[swap] = swap.rate(s_cv).real
fwd_swaps

risk = {}
for swap, rate in fwd_swaps.items():
    risk.update({swap.end: swap.risk(s_cv, fixed_rate=rate)[:, 0]})

df = pd.DataFrame(risk, index=["1y", "2y", "5y", "10y"])
df.style.format("{:.3f}")

What we have done above is to risk forward swaps against our curve. When we scale each of these columns to one we have effectively built a Jacobian transformation for forward swaps to par swaps.

In [ ]:
J_fwd_par = (df / df.sum()).to_numpy()
pd.DataFrame(J_fwd_par).style.format("{:.3f}")

### Par to Forward Jacobian

Lets consider the process in reverse, defining our curve from forward swaps.

In [ ]:
s_cv = SolvedCurve(
    nodes=nodes,
    swaps=list(fwd_swaps.keys()),
    obj_rates=list(fwd_swaps.values()),
    interpolation="log_linear",
    algorithm="levenberg_marquardt"
)
print(s_cv.iterate())

In [ ]:
for swap in fwd_swaps.keys():
    print(swap.rate(s_cv).real)

In [ ]:
risk = {}
for swap in fwd_swaps.keys():
    risk.update({swap.end: swap.risk(s_cv, fixed_rate=swap.rate(s_cv).real)[:, 0]})
    
df = pd.DataFrame(risk, index=["1y", "1y1y", "2y3y", "5y5y"])
df.style.format("{:.3f}")

In [ ]:
risk = {}
for swap, rate in swaps.items():
    risk.update({swap.end: swap.risk(s_cv, fixed_rate=rate)[:, 0]})

df = pd.DataFrame(risk, index=["1y", "1y1y", "2y3y", "5y5y"])
df.style.format("{:.3f}")

What we have done above is to risk par swaps against our forward curve. When we scale each of these columns to one we have effectively built a Jacobian transformation for par swaps to forward swaps.

In [ ]:
J_par_fwd = (df / df.sum()).to_numpy()
pd.DataFrame(J_par_fwd).style.format("{:.3f}")

### Testing the lossless nature of our numerical library

We should be able to convert and then convert back to obtain the same risks.

Suppose we convert 1000 10Y par risk to fwd risk and then back to par risk, what happens?

In [ ]:
par_risk = np.array([0, 0, 0, 1000])[:, np.newaxis]
fwd_risk = np.matmul(J_par_fwd, par_risk)
fwd_risk

In [ ]:
par_risk_reversed = np.matmul(J_fwd_par, fwd_risk)
par_risk_reversed

This is close, but we have lost some precision due to either machine precision or numerical truncation.

The same can be seen if we compare the analytical inverses of the Jacobians.

In [ ]:
pd.DataFrame(J_par_fwd - np.linalg.inv(J_fwd_par)).style.format("{:.4f}")

In [ ]:
pd.DataFrame(J_fwd_par - np.linalg.inv(J_par_fwd)).style.format("{:.4f}")